In [ ]:
""" import settings """
%load_ext autoreload
%autoreload 2

import numpy as np
from matplotlib import pyplot as plt
from matplotlib_settings import set_plot_settings, reset_plot_settings

# Set the plot settings
set_plot_settings()

# import global variables
from utils_motor_global import *
from utils_motor_sigproc import get_mt_ch_psd

In [ ]:
""" load channel data """
load_dir = CH_DATA_DIR
motion_t    = np.load(f'{load_dir}/motion_t.npy')
wrist_vel_y = np.load(f'{load_dir}/wrist_vel_y.npy')

t           = np.load(f'{load_dir}/t.npy')
ch_data_HB  = np.load(f'{load_dir}/ch_data.npy')
ch_data     = np.load(f'{load_dir}/ch_data_HB_removed.npy')

spect_t     = np.load(f'{load_dir}/spect_t.npy')
spect_f     = np.load(f'{load_dir}/spect_f.npy')
ch_zs_spect = np.load(f'{load_dir}/ch_zs_spect.npy')

In [ ]:
""" convert to input-referred voltage """
ch_data = ch_data*FS_ADC/MAX_ADC_CODE/GAIN
ch_data_HB = ch_data_HB*FS_ADC/MAX_ADC_CODE/GAIN

Plot Example Channel Spectrogram, Time-domain Waveform

In [ ]:
# pick length of segment to plot
t_start, t_end = 0, 20 # seconds

In [ ]:
""" plot motion, time domain trace, spectrogram of a single channel """
r, c = 3, 8
# ch = 16*r + c # pick a channel

# spectrogram params
ch_zs_spect = ch_zs_spect[::16, :] # downsample by 16
t0, t1 = t_start, t_end
f0, f1 = int(spect_f[0]), round(spect_f[-1])
extent = [t0, t1, f1, f0]
q_cut = 0.01
vmin = np.quantile(ch_zs_spect, q_cut)
vmax = np.quantile(ch_zs_spect, 1-q_cut)
cmap = 'viridis'

# plot
plt.close('all')
k = 0.9
fig, ax = plt.subplots(3, 1, sharex=True, figsize=(10*k, 6*k),
                       gridspec_kw = {'height_ratios': [1.5, 1.5, 2]})
plt.subplots_adjust(hspace=0.3)

ax[0].plot(motion_t, wrist_vel_y)
ax[1].plot(t, ch_data/1e-6)

im = ax[2].imshow(ch_zs_spect.T, aspect='auto', vmin=vmin, vmax=vmax, extent=extent, cmap=cmap,
             interpolation='bilinear')
ax[2].invert_yaxis()


ax[0].set_yticks([0])
ax[2].set_xticks([0, 5, 10, 15, 20])

for ii in range(3):
    ax[ii].grid(True)

ax[0].set_title(f'Channel {r, c}\n')
fontsize = 14
ax[0].set_ylabel('Wrist y-vel.\n(normalized)\n', fontsize=fontsize)
ax[1].set_ylabel('Recording (μV)', fontsize=fontsize)
ax[2].set_ylabel('Frequency (Hz)', fontsize=fontsize)

ax[2].set_xlabel('Time (s)')

left_cbar = 0.92  # Adjust left position of the colorbars
bottom_cbar0 = 0.11  # Adjust bottom position of the colorbars
width_cbar = 0.015  # Adjust width of the colorbars
height_cbar = 0.26  # Adjust height of the colorbars

cbar0_ax = fig.add_axes([left_cbar, bottom_cbar0, width_cbar, height_cbar])
cbar0 = fig.colorbar(im, ax=ax[2], cax=cbar0_ax)
# cbar0.set_ticks([-1.5, 0, 1.5])

Plot Example Channel Power Spectral Density, Before and After Hemodynamics Removal

In [ ]:
""" multi-taper params """
from scipy.signal.windows import dpss

# pick length of segment to plot
t_start, t_end = 0, 20 # seconds

Ts = t[1] - t[0]
rec_idx0 = np.where(t >= t_start)[0][0]
rec_idx1 = np.where(t <= t_end)[0][-1]
len_win = t[rec_idx1] - t[rec_idx0]

# pick half-BW
W = 0.5 # Hz. frequency smoothing: [f0 - W , f0 + W]

NW = len_win*W # common choices are 2.5, 3, 3.5, 4
K = int(2*NW - 1) # Number of tapers
print(f'{NW=:.2f}, {K=}')

# dependent params
fs = 1/Ts
win_size = rec_idx1 - rec_idx0 + 1
n = win_size
half_n = int(np.ceil(win_size/2))
freq = np.fft.fftfreq(win_size, d = 1/fs)
half_freq = freq[:half_n]
fbin = half_freq[1]

# DPSS
dpss_tapers, dpss_eigen = dpss(n, NW, K, return_ratios=True)
wt = np.ones(K)/K # apply unity weight

In [ ]:
""" plot a single channel PSD """
assert len(ch_data) == win_size

ch_psd_HB = get_mt_ch_psd(ch_data_HB, dpss_tapers, wt)
ch_psd    = get_mt_ch_psd(ch_data, dpss_tapers, wt)

# normalize for plotting. plot LFP range: (1 to 300) Hz
psd_idx0 = np.where(half_freq >= 1)[0][0]
psd_idx1 = np.where(half_freq <= 300)[0][-1] + 1
norm_val = np.max(ch_psd_HB[psd_idx0:psd_idx1])
ch_psd_HB = ch_psd_HB/norm_val
ch_psd    = ch_psd/norm_val

# plot
plt.close('all')
fig, ax = plt.subplots(figsize=(4, 3))
ax.loglog(half_freq, ch_psd_HB)
ax.loglog(half_freq, ch_psd)

ax.set_xlim((1, 300))
# ax.set_ylim((np.min(ch_psd)*0.5, 2))
ax.set_ylim((1e-5, 2))
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('PSD (a.u.)')

ax.set_title(f'Channel {r, c}')
ax.grid(True)

ax.axvline(x = HB_BPF_LOW , linestyle='--', color='k')
ax.axvline(x = HB_BPF_HIGH, linestyle='--', color='k')

ax.legend(['Before HD Removal', 'After HD Removal'], loc = (1.02, 0))